In [1]:
def reading(f_name):

    TS = [[],[],[]]
    FS = [[],[],[]]
    PS = [[],[],[],[]]
    
    f = open(f_name, 'r')
    
    for line in f:
        if 'Grasp' in line:
            g = int(line[6:]) - 1
            for i in range(3):
                TS[i].append([])
                FS[i].append([])
                PS[i].append([])
            PS[3].append([])
        else:
            num = int(line[2])
            content = line[4:]
            if 'TS' in line:
                TS[num][g].append(list(eval(content))[2])
            if 'FS' in line:
                FS[num][g].append(list(eval(content))[2])
            if 'PS' in line:
                PS[num][g].append(list(eval(content))[2])
                
    f.close()
    return(TS, FS, PS)



In [2]:
import os

path = './Data/data'

for root, dirs, files in os.walk(path):
    f_ls = files


In [3]:
import numpy as np

clt = np.empty(len(f_ls) * 10 * 200 * 300).reshape((len(f_ls), 10, 200, 300))
root = './Data/data/'
i = 0

for f_name in f_ls:
    path = root + f_name
    print(f_name)
    
    TS, FS, PS = reading(path)
    clt[i] = np.vstack((np.array(TS), np.array(FS), np.array(PS)))
    i += 1

apple.txt
cleanser_bottle.txt
cuboid(rigid).txt
cuboid(soft).txt
cup(rigid).txt
cup(soft).txt
lemon.txt
orange.txt
sphere(rigid).txt
sphere(soft).txt


In [419]:
def finger_tip_sensor_handling(ary):
    
    features = []
    
    for i in range(3):
        features.append(np.min(ary[i]))
        
    features += ary[:, 20:120].mean(1).tolist()
    features += ary[:, 150:250].mean(1).tolist()
    features += ary[:, 250:280].mean(1).tolist()
        
    unique,count = np.unique(np.abs(ary[1]) > 2, return_counts=True)
    if len(unique) == 2:
        features.append(count[1]/100)
    else:
        features.append(0)
    
    return(features)

def finger_pulp_sensor_handling(ary):
    
    features = 0
    
    mean = ary.mean(1)
    for i in range(3):
        unique,count = np.unique((ary[i]>mean[i]+3) | (ary[i]<mean[i]-3), return_counts=True)
        if len(unique) == 2:
            features += (count[1]/100)

    return(features)

def proximity_sensor_handling(ary):
    
    features = []
    mx = [-2, -2 ,-2]
    mx_pos = [50, 50, 50]
    mn = [-1, -1, -1]
    mn_pos = [0, 0, 0]
    mean = [0, 0, 0]
    p4_mn = [0]
    
    for i in range(3):
        minm = 1
        pos = 0
        mx[i] = np.max(ary[i, :50])
        mx_pos[i] = np.where(a==m)[0][0]
        
        for j in range(50):
            if (ary[i, j] < minm) and (ary[i, j] > 0.0025):
                minm = ary[i,j]
                pos = j
        mn[i] = minm
        mn_pos[i] = pos
        if (mx_pos[i] < mn_pos[i]) and (mx[i] > mn[i]):
            mean[i] = ary[i, mx_pos[i]:mn_pos[i]].mean()
    
    p4_mn[0] = ary[3, :250].mean()
    
    return(mn+p4_mn)
        
        
    

In [420]:
obj_num = clt.shape[0]
grasp_num = clt.shape[2]
feature_num = 13 + 1 + 4
X = np.empty(obj_num*grasp_num*feature_num).reshape(obj_num*grasp_num, feature_num)
Y = np.empty(obj_num*grasp_num)

for i in range(obj_num):
    for s in range(grasp_num):
        tip_sensor_feature = np.array(finger_tip_sensor_handling(clt[i, :3, s, :]))
        pulp_sensor_feature = np.array(finger_pulp_sensor_handling(clt[i, 3:6, s, :]))
        proximity_sensor_feature = np.array(proximity_sensor_handling(clt[i, 6:, s, :]))
        X[i * grasp_num + s] = np.hstack((tip_sensor_feature, pulp_sensor_feature, proximity_sensor_feature))
       # X[i * grasp_num + s] = proximity_sensor_feature
        Y[i * grasp_num + s] = i
    

In [457]:
import numpy as np
from sklearn import linear_model, datasets, preprocessing
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
logreg = linear_model.LogisticRegression(C=1, max_iter = 20000)

X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)

logreg.fit(X_train_scaled, Y_train)
 

prepro = logreg.predict_proba(X_test_scaled)
Y_pre = logreg.predict(X_test_scaled)
acc = logreg.score(X_test_scaled,Y_test)

In [458]:
obj = []
obj_ = []
for i in range(len(f_ls)):
    obj.append(0)
    obj_.append(0)
            
for i in range(len(Y_test)):
    if Y_test[i] == Y_pre[i]:
        obj_[int(Y_test[i])] += 1
    obj[int(Y_test[i])] += 1
    
print('Accuracy:')
for i in range(len(f_ls)):
    print(f_ls[i][:-4], '###', obj_[i], '/', obj[i], '=', obj_[i]/obj[i])
print('Overall:', acc)
    


Accuracy:
apple ### 27 / 46 = 0.5869565217391305
cleanser_bottle ### 40 / 42 = 0.9523809523809523
cuboid(rigid) ### 17 / 41 = 0.4146341463414634
cuboid(soft) ### 28 / 37 = 0.7567567567567568
cup(rigid) ### 30 / 34 = 0.8823529411764706
cup(soft) ### 15 / 40 = 0.375
lemon ### 26 / 44 = 0.5909090909090909
orange ### 15 / 34 = 0.4411764705882353
sphere(rigid) ### 31 / 44 = 0.7045454545454546
sphere(soft) ### 9 / 38 = 0.23684210526315788
Overall: 0.595


In [464]:
import copy

obj_ls = copy.deepcopy(f_ls)
for i in range(len(obj_ls)):
    obj_ls[i] =  obj_ls[i][:-4]
obj_ls += ['cuboid', 'cup', 'sphere']

obj = []
obj_ = []
for i in range(len(obj_ls)):
    obj.append(0)
    obj_.append(0)

for i in range(len(Y_test)):
    if Y_test[i] in [2,3]:
        obj[10] += 1
    if Y_test[i] in [4,5]:
        obj[11] += 1
    if Y_test[i] in [8,9]:
        obj[12] += 1
    if Y_pre[i]  in [2,3]:
        obj_[10] += 1
    if Y_pre[i] in [4,5]:
        obj_[11] += 1
    if Y_pre[i] in [8,9]:
        obj_[12] += 1
    if Y_test[i] == Y_pre[i]:
        obj_[int(Y_test[i])] += 1
    obj[int(Y_test[i])] += 1
    
for i in range(len(f_ls),len(obj_ls)):
    print(obj_ls[i], '###', obj_[i], '/', obj[i], '=', obj_[i]/obj[i])
 


cuboid ### 76 / 78 = 0.9743589743589743
cup ### 74 / 74 = 1.0
sphere ### 72 / 82 = 0.8780487804878049
